In [1]:
import copy

In [2]:
def check(state, goalState):
    wildcards = {}
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            state_block_ids = [y.block_id for y in state]
            
            if(ary[0] == "has" and ary[1][:8] != "wildcard"):
                if(ary[1] not in state_block_ids):
                    print("Block in goal isn't in state")
                    return False
                if(len(x.split()) == 6):
                    temp = find(ary[3], ary[4], ary[5], state)
                    if(temp == None or temp.block_id != ary[1]):
                        return False
            if(ary[0] == "is" and ary[2][:8] != "wildcard" and ary[1][:8] != "wildcard"):
                if((ary[2] not in state_block_ids) or (ary[1] not in state_block_ids)):
                        print("Block in goal isn't in state")
                        return False
                if(ary[3] == "on-top-of"):
                        for z in state:
                            if(z.block_id == ary[2] and (z.top.block_id if z.top != None else None) != ary[1]):
                                return False

                if(ary[3] == "side-by-side"):
                    for z in state:
                        if(z.block_id == ary[2] and (ary[1] not in [w.block_id for w in z.sides])):
                            return False
                        
       
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 4):
                    no_color = True
                    for z in startState:
                        if(z.color == ary[3]):
                            no_color = False
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
                    if(no_color):
                        print("There is no block that is color " + ary[3])
                        return False
                if(len(x.split()) == 6):
                    temp = find(ary[3], ary[4], ary[5], state)
                    if(temp == None):
                        return False
                    elif(wildcards.get(ary[1], 0) == 0):
                        wildcards[ary[1]] = [temp]
                    else:
                        remove = []
                        for y in wildcards[ary[1]]:
                            if(y != temp):
                                remove = remove + [y]
                        for r in remove:
                            wildcards[ary[1]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
            if(ary[0] == "is" and (ary[2][:8] == "wildcard" or ary[1][:8] == "wildcard")):
                if(ary[3] == "on-top-of" and ary[2][:8] != "wildcard"):
                    block_two = next((a for a in state if a.block_id == ary[2]), None)
                    if(block_two.top == None):
                        return False
                    elif(wildcards.get(ary[1], 0) == 0):
                        wildcards[ary[1]] = [block_two.top]
                    else:
                        remove = []
                        for w in wildcards[ary[1]]:
                            if(w.below != block_two):
                                remove = remove + [w]
                        for r in remove:
                            wildcards[ary[1]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                elif(ary[3] == "on-top-of" and ary[1][:8] != "wildcard"):
                    block_one = next((a for a in state if a.block_id == ary[1]), None)
                    if(block_one.below == None):
                        return False
                    elif(wildcards.get(ary[2], 0) == 0):
                        wildcards[ary[2]] = [block_one.below]
                    else:
                        remove = []
                        for w in wildcards[ary[2]]:
                            if(w.top != block_one):
                                remove = remove + [w]
                        for r in remove:
                            wildcards[ary[2]].remove(r)
                        if(wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0):
                            return False
                elif(ary[3] == "on-top-of"):
                    if(wildcards.get(ary[1], 0) == 0 and wildcards.get(ary[2], 0) == 0):
                        for v in state:
                            if(v.top != None):
                                if(wildcards.get(ary[1], 0) == 0): 
                                    wildcards[ary[1]] = [v.top]
                                else:
                                    wildcards[ary[1]].append(v.top)
                                if(wildcards.get(ary[2], 0) == 0): 
                                    wildcards[ary[2]] = [v]
                                else:
                                    wildcards[ary[2]].append(v)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                    elif(wildcards.get(ary[2], 0) == 0):
                        remove = []
                        for w in wildcards[ary[1]]:
                            if(w.below == None):
                                remove = remove + [w]
                            else:
                                if(wildcards.get(ary[2], 0) == 0):
                                    wildcards[ary[2]] = [w.below]
                                else:
                                    wildcards[ary[2]].append(w.below)
                        for r in remove:
                            wildcards[ary[1]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                    elif(wildcards.get(ary[1], 0) == 0):
                        remove = []
                        for w in wildcards[ary[2]]:
                            if(w.top == None):
                                remove = remove + [w]
                            else:
                                if(wildcards.get(ary[1], 0) == 0):
                                    wildcards[ary[1]] = [w.top]
                                else:
                                    wildcards[ary[1]].append(w.top)
                        for r in remove:
                            wildcards[ary[2]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False  
                    else:
                        remove = []
                        for m in wildcards[ary[1]]:
                            not_related_1 = True
                            for n in wildcards[ary[2]]:
                                if(m.below == n):
                                    not_related_1 = False
                            if(not_related_1):
                                remove = remove + [m]
                        for r in  remove:
                            wildcards[ary[1]].remove(r)

                        remove = []
                        for m in wildcards[ary[2]]:
                            not_related_2 = True
                            for n in wildcards[ary[1]]:
                                if(m.top == n):
                                    not_related_2 = False
                            if(not_related_2):
                                remove = remove + [m]
                        for r in remove:
                            wildcards[ary[2]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False



                elif(ary[3] == "side-by-side" and ary[2][:8] != "wildcard"):
                    block_two = next((a for a in state if a.block_id == ary[2]), None)
                    if(len(block_two.sides) == 0):
                        return False
                    elif(wildcards.get(ary[1], 0) == 0):
                        for j in block_two.sides:
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [block_two]
                            else:
                                wildcards[ary[1]].append(block_two)
                        if(wildcards.get(ary[1], 0) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                    else:
                        remove = []
                        for w in wildcards[ary[1]]:
                            if(w not in block_two.sides):
                                remove = remove + [w]
                        for r in remove:
                            wildcards[ary[1]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                elif(ary[3] == "side-by-side" and ary[1][:8] != "wildcard"):
                    block_one = next((a for a in state if a.block_id == ary[1]), None)
                    if(len(block_one.sides) == 0):
                        return False
                    elif(wildcards.get(ary[2], 0) == 0):
                        for j in block_one.sides:
                            if(wildcards.get(ary[2], 0) == 0):
                                wildcards[ary[2]] = [block_one]
                            else:
                                wildcards[ary[2]].append(block_one)
                        if(wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0):
                                return False
                    else:
                        remove = []
                        for w in wildcards[ary[2]]:
                            if(w not in block_one.sides):
                                remove = remove + [w]
                        for r in remove:
                             wildcards[ary[2]].remove(2)
                        if(wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0):
                            return False
                elif(ary[3] == "side-by-side"):
                    if(wildcards.get(ary[1], 0) == 0 and wildcards.get(ary[2], 0) == 0):
                        for v in state:
                            if(len(v.sides) > 0):
                                if(wildcards.get(ary[1], 0) == 0): 
                                    wildcards[ary[1]] = [v]
                                else:
                                    wildcards[ary[1]].append(v)
                                for k in v.sides:
                                    if(wildcards.get(ary[2], 0) == 0): 
                                        wildcards[ary[2]] = [k]
                                    else:
                                        wildcards[ary[2]].append(k)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                    elif(wildcards.get(ary[2], 0) == 0):
                        remove = []
                        for w in wildcards[ary[1]]:
                            if(len(w.sides) == 0):
                                remove = remove + [w] 
                            else:
                                for p in w.sides:
                                    if(wildcards.get(ary[2], 0) == 0):
                                        wildcards[ary[2]] = [p]
                                    else:
                                        wildcards[ary[2]].append(p)
                        for r in remove:
                            wildcards[ary[1]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                    elif(wildcards.get(ary[1], 0) == 0):
                        remove = []
                        for w in wildcards[ary[2]]:
                            if(len(w.sides) == 0):
                                remove = remove + [w] 
                            else:
                                for p in w.sides:
                                    if(wildcards.get(ary[1], 0) == 0):
                                        wildcards[ary[1]] = [p]
                                    else:
                                        wildcards[ary[1]].append(p)
                        for r in remove:
                            wildcards[ary[2]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
                    else:
                        remove = []
                        for m in wildcards[ary[1]]:
                            not_related_1 = True
                            for n in wildcards[ary[2]]:
                                if(n in m.sides):
                                    not_related_1 = False
                            if(not_related_1):
                                remove = remove + [m]
                        for r in remove:
                            wildcards[ary[1]].remove(r)
                        remove = []
                        for m in wildcards[ary[2]]:
                            not_related_2 = True
                            for n in wildcards[ary[1]]:
                                if(n in m.sides):
                                    not_related_2 = False
                            if(not_related_2):
                                remove = remove + [m]
                        for r in remove:
                            wildcards[ary[2]].remove(r)
                        if(wildcards.get(ary[1], 0) == 0 or wildcards.get(ary[2], 0) == 0 or len(wildcards[ary[2]]) == 0 or len(wildcards[ary[1]]) == 0):
                            return False
    return True

In [3]:
class block:
    
    def __init__(self, block_id, x, y, z, color = None):                             
        self.block_id = block_id
        self.x = int(x)
        self.y = int(y)
        self.z = int(z)
        self.color = color
        self.top = None
        self.below = None
        self.sides = []
        self.grabbed = False
        
    def __lt__(self, other):
         return int(self.block_id[5:]) < int(other.block_id[5:])
        
    def __eq__(self, other):
        if(other == "table" or other == None):
            return False
        else:
            return self.block_id == other.block_id
        
    def __ne__(self, other):
        if(other == "table" or other == None):
            return True
        else:
            return self.block_id != other.block_id
        
    def set_top(self, block_one):
        if(self != block_one.top):
            if(self.below != None): self.below.top = None
            self.below = block_one
            block_one.top = self
    
    def remove_bottom(self):
        if(self.below != None): self.below.top = None
        self.below = None
    
    def add_side(self, block_one):
        if(self not in block_one.sides):
            self.sides.append(block_one)
            block_one.sides.append(self)
        
    def remove_sides(self):
        for s in self.sides:
            s.sides.remove(self)
        self.sides = []
        
    def __repr__(self):
        return "{" + self.block_id +", ("+str(self.x) + ", " + str(self.y) + ", " + str(self.z) + "), (" + \
        "color: " + ("None", self.color)[self.color != None] + "), (on-top-of: " + \
        ("None" if self.top == None else self.top.block_id) + "), (side-by-side: " + \
        str([x.block_id for x in self.sides]) + ")}\n"
        
    def __str__(self):
        return "{" + self.block_id +", ("+str(self.x) + ", " + str(self.y) + ", " + str(self.z) + "), (" + \
        "color: " + ("None", self.color)[self.color != None] + "), (on-top-of: " + \
        ("None" if self.top == None else self.top.block_id) + "), (side-by-side: " + \
        str([x.block_id for x in self.sides]) + ")}\n"

In [4]:
def grabbed(state):
    if(True in [x.grabbed for x in state]):
        return True
    return False

In [5]:
def find(x, y, z, state):
    return next((a for a in state if (a.x == int(x) and a.y == int(y) and a.z == int(z))), None)

In [6]:
def find2(block_id, state):
    return next((a for a in state if (a.block_id == block_id)), None)

In [7]:
# block_one = block("block1", 0, 0, 0)
# block_two = block("block2", 1, 1, 0)
# block_three = block("block3", 1, 3, 0)
# block_four = block("block4", 1, 4, 0)
# state = [block_one, block_two, block_three, block_four]
# print(state)
# grab(block_one, state)
# carry(block_one, 1, 1, 1, state)
# release(block_one)
# grab(block_four, state)
# carry(block_four, 0, -1, 1, state)
# release(block_four)
# print(state)
# slide(block_three, 0, -1, state)
# print(state)
# slide(block_three, 0, 1, state)
# print(state)
# grab(block_four, state)
# carry(block_four, 0, 1, -1, state)
# release(block_four)
# print(state)

In [8]:
def grab(block_one, state):
    if(grabbed(state)):
        print("Cannot grab! A block is already grabbed")
    elif(block_one.top != None):
        print("Cannot grab! " + block_one.block_id + " has a block on top of it")
    else:
        block_one.grabbed = True

In [9]:
def get_grabbed(state):
    for inds, s in enumerate(state):
        if(s.grabbed == True):
            return (inds, s)
    return (-1, None)

In [10]:
def release(block_one, state):
    if(block_one.grabbed == False):
        print("Cannot release! " + block_one.block_id + " is not currently grabbed")
    elif(block_one.z > 0 and (find(block_one.x, block_one.y, block_one.z - 1, state) == None)):
        print("Cannot release! There is no block underneath that spot")
    else:
        block_one.grabbed = False

In [11]:
def carry(block_one, deltax, deltay, deltaz, state):
    if(block_one.grabbed == False):
        print("Cannot carry! " + block_one.block_id + " is not currently grabbed")
    elif(find(block_one.x + deltax, block_one.y + deltay, block_one.z + deltaz, state) != None):
        print("Cannot carry! There is already a block in spot " + str((block_one.x + deltax, block_one.y + deltay, block_one.z + deltaz)))
    else:
        block_one.x = block_one.x + deltax
        block_one.y = block_one.y + deltay
        block_one.z = block_one.z + deltaz
        block_one.remove_bottom()
        block_one.remove_sides()
        zdown = find(block_one.x, block_one.y, block_one.z - 1, state)
        xup = find(block_one.x + 1, block_one.y, block_one.z, state)
        xdown = find(block_one.x - 1, block_one.y, block_one.z, state)
        yup = find(block_one.x, block_one.y + 1, block_one.z, state)
        ydown = find(block_one.x, block_one.y - 1, block_one.z, state)
        if(zdown != None): block_one.set_top(zdown)
        if(xup != None): block_one.add_side(xup)
        if(xdown != None): block_one.add_side(xdown)
        if(yup != None): block_one.add_side(yup)
        if(ydown != None): block_one.add_side(ydown)
            

In [12]:
def slide(block_one, deltax, deltay, state):
    if(grabbed(state)):
        print("Cannot slide! A block is grabbed")
    elif(block_one.z != 0):
        print("Cannot slide! " + block_one.block_id + " is not at height 0")
    elif(find(block_one.x + deltax, block_one.y + deltay, block_one.z, state) != None):
        print("Cannot slide! There is already a block in spot " + str((block_one.x + deltax, block_one.y + deltay, block_one.z)))
    else:
        block_one.x = block_one.x + deltax
        block_one.y = block_one.y + deltay
        block_one.remove_sides()
        xup = find(block_one.x + 1, block_one.y, block_one.z, state)
        xdown = find(block_one.x - 1, block_one.y, block_one.z, state)
        yup = find(block_one.x, block_one.y + 1, block_one.z, state)
        ydown = find(block_one.x, block_one.y - 1, block_one.z, state)
        if(xup != None): block_one.add_side(xup)
        if(xdown != None): block_one.add_side(xdown)
        if(yup != None): block_one.add_side(yup)
        if(ydown != None): block_one.add_side(ydown)
        temp = block_one
        while(temp.top != None):
            temp = temp.top
            temp.x = temp.x + deltax
            temp.y = temp.y + deltay
            temp.remove_sides()
            xup = find(temp.x + 1, temp.y, temp.z, state)
            xdown = find(temp.x - 1, temp.y, temp.z, state)
            yup = find(temp.x, temp.y + 1, temp.z, state)
            ydown = find(temp.x, temp.y - 1, temp.z, state)
            if(xup != None): temp.add_side(xup)
            if(xdown != None): temp.add_side(xdown)
            if(yup != None): temp.add_side(yup)
            if(ydown != None): temp.add_side(ydown)
            

In [13]:
def read_startState(file_name):
    f = list(open(file_name, "r"))
    state = []
    for x in f:
        if(len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has"):
                new_block = block(ary[1], ary[3], ary[4], ary[5])
                state.append(new_block)
    for s in state:
        zup = find(s.x, s.y, s.z + 1, state)
        zdown = find(s.x, s.y, s.z - 1, state)
        xup = find(s.x + 1, s.y, s.z, state)
        xdown = find(s.x - 1, s.y, s.z, state)
        yup = find(s.x, s.y + 1, s.z, state)
        ydown = find(s.x, s.y - 1, s.z, state)
        if(zup != None): zup.set_top(s)
        if(zdown != None): s.set_top(zdown)
        if(xup != None): s.add_side(xup)
        if(xdown != None): s.add_side(xdown)
        if(yup != None): s.add_side(yup)
        if(ydown != None): s.add_side(ydown)
    for z in f:
        if(len(z.split()) == 4):
            ary = z.replace("(", "").replace(")", "").split()
            if(ary[0] == "has"):
                block_one = next((a for a in state if (a.block_id == ary[1])), None)
                block_one.color = ary[3]
                
    return sorted(state)

In [14]:
def read_goalState(file_name):
    return sorted(list(open(file_name, "r")))

In [15]:
startState = read_startState("input1.txt")
print(startState)
goalState = read_goalState("output1.txt")
print(check(startState, goalState))
# print(startState)
# grab(startState[5], startState)
# carry(startState[5], 0, 0, 1, startState)
# release(startState[5], startState)
# carry(startState[5], 0, 0, 1, startState)
# release(startState[5], startState)
# carry(startState[5], 0, -1, 1, startState)
# release(startState[5], startState)
# print(startState)

[{block1, (1, 1, 0), (color: green), (on-top-of: block2), (side-by-side: ['block6'])}
, {block2, (1, 1, 1), (color: blue), (on-top-of: block3), (side-by-side: [])}
, {block3, (1, 1, 2), (color: red), (on-top-of: None), (side-by-side: [])}
, {block4, (1, 3, 0), (color: blue), (on-top-of: block5), (side-by-side: ['block6'])}
, {block5, (1, 3, 1), (color: red), (on-top-of: None), (side-by-side: [])}
, {block6, (1, 2, 0), (color: green), (on-top-of: None), (side-by-side: ['block1', 'block4'])}
, {block7, (8, 8, 0), (color: purple), (on-top-of: None), (side-by-side: [])}
, {block8, (5, 5, 0), (color: yellow), (on-top-of: None), (side-by-side: [])}
]
True


In [16]:
def actions(state):
    actions = []
    slides = [(-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1), (-1,0)]
    carries = [(-1,1,0), (0,1,0), (1,1,0), (1,0,0), (1,-1,0), (0,-1,0), (-1,-1,0), (-1,0,0), (-1,1,1), (0,1,1), (1,1,1), (1,0,1), (1,-1,1), (0,-1,1), (-1,-1,1), (-1,0,1), (0,0,1), (-1,1,-1), (0,1,-1), (1,1,-1), (1,0,-1), (1,-1,-1), (0,-1,-1), (-1,-1,-1), (-1,0,-1), (0,0,-1)]


    cost = 1
    grabbed_block = (-1, None)
    if_grabbed = grabbed(state)
    if(if_grabbed): 
        cost = 2
        grabbed_block = get_grabbed(state)

    if(cost == 1 or (grabbed_block[1].z == 0 or grabbed_block[1].below != None)):
        for inda, a in enumerate(state):
            if(a.z == 0):
                for s in slides:
                    if(9 >= a.x + s[0] >= 0 and 9 >= a.y + s[1] >= 0 and find(a.x + s[0], a.y + s[1], 0, state) == None):
                        actions.append(("slide" + ("-r" if cost == 2 else ""), (inda, a.block_id), (grabbed_block[0], (grabbed_block[1].block_id if grabbed_block[1] != None else None)), s, cost))
  
    for indb, b in enumerate(state):
        cost = 1
        if(b.top == None and (grabbed_block[1] == None or grabbed_block[1] == b or (grabbed_block[1].z == 0 or grabbed_block[1].below != None))):
            if(grabbed_block[1] == None):
                cost = 2
            elif(grabbed_block[1] == b):
                cost = 1
            else:
                cost = 3
            for c in carries:
                    if(9 >= b.x + c[0] >= 0 and 9 >= b.y + c[1] >= 0 and find(b.x + c[0], b.y + c[1], b.z + c[2], state) == None):
                        actions.append(("carry" + ("-g" if cost == 2 else "") + ("-rg" if cost == 3 else ""), (indb, b.block_id), (grabbed_block[0],(grabbed_block[1].block_id if grabbed_block[1] != None else None)), c, cost))
    
    return actions

In [17]:
def take_action(action, state):
    newState = copy.deepcopy(state)
    if(action[0] == "slide"):
        slide(newState[action[1][0]], action[3][0], action[3][1], newState)
    elif(action[0] == "slide-r"):
        release(newState[action[2][0]], newState)
        slide(newState[action[1][0]], action[3][0], action[3][1], newState)
    elif(action[0] == "carry"):
        carry(newState[action[1][0]], action[3][0], action[3][1], action[3][2], newState)
    elif(action[0] == "carry-g"):
        grab(newState[action[1][0]], newState)
        carry(newState[action[1][0]], action[3][0], action[3][1], action[3][2], newState)
    elif(action[0] == "carry-rg"):
        release(newState[action[2][0]], newState)
        grab(newState[action[1][0]], newState)
        carry(newState[action[1][0]], action[3][0], action[3][1], action[3][2], newState)
 
    return newState

In [18]:
startState = read_startState("input2.txt")
print(startState)
# grab(startState[0], startState)
# carry(startState[0], 0, 0, 1, startState)
# print(actions(startState))\
for x in actions(startState):
    print(x)
# take_action(actions(startState)[0], startState)


[{block1, (1, 1, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (3, 3, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
]
('slide', (0, 'block1'), (-1, None), (-1, 1), 1)
('slide', (0, 'block1'), (-1, None), (0, 1), 1)
('slide', (0, 'block1'), (-1, None), (1, 1), 1)
('slide', (0, 'block1'), (-1, None), (1, 0), 1)
('slide', (0, 'block1'), (-1, None), (1, -1), 1)
('slide', (0, 'block1'), (-1, None), (0, -1), 1)
('slide', (0, 'block1'), (-1, None), (-1, -1), 1)
('slide', (0, 'block1'), (-1, None), (-1, 0), 1)
('slide', (1, 'block2'), (-1, None), (-1, 1), 1)
('slide', (1, 'block2'), (-1, None), (0, 1), 1)
('slide', (1, 'block2'), (-1, None), (1, 1), 1)
('slide', (1, 'block2'), (-1, None), (1, 0), 1)
('slide', (1, 'block2'), (-1, None), (1, -1), 1)
('slide', (1, 'block2'), (-1, None), (0, -1), 1)
('slide', (1, 'block2'), (-1, None), (-1, -1), 1)
('slide', (1, 'block2'), (-1, None), (-1

In [19]:
def h_simple(state, goal):
    return 1

In [20]:
def h_1(state, goal):
    cost = 0
    dic = {}
    wildcards = {}
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 4):
                    for z in startState:
                        if(z.color == ary[3]):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
                if(len(x.split()) == 6):
                    if(wildcards.get(ary[1], 0) == 0):
                        for z in startState:
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
            if(ary[0] == "has" and ary[1][:8] != "wildcard" and len(x.split()) == 6 and dic.get(ary[1], 0) == 0):
                temp = find2(ary[1], state)
                dif = [abs(temp.x - int(ary[3])), abs(temp.y - int(ary[4]))]
                cost = cost + max(dif)
                if(a.grabbed != True):
                    a = copy.deepcopy(temp)
                    while(a.top != None and a.top.grabbed == False):
                        a = a.top
                        dic[a.block_id] = 1 
                    a = copy.deepcopy(temp)
                    while(a.below != None and a.below.grabbed == False):
                        a = a.below
                        dic[a.block_id] = 1
            if(ary[0] == "has" and ary[1][:8] == "wildcard" and len(x.split()) == 6):
                if(wildcards.get(ary[1], 0) != 0):
                    min_card = (None, 1000)
                    for w in wildcards[ary[1]]:
                        dif = [abs(temp.x - int(ary[3])), abs(temp.y - int(ary[4]))]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                    if(dic.get(min_card[0].block_id, 0) == 0):
                        cost = cost + min_card[1]
                        if(min_card[0].grabbed != True):
                            a = copy.deepcopy(min_card[0])
                            while(a.top != None and a.top.grabbed == False):
                                a = a.top
                                dic[a.block_id] = 1 
                            a = copy.deepcopy(min_card[0])
                            while(a.below != None and a.below.grabbed == False):
                                a = a.below
                                dic[a.block_id] = 1

                                
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] != "wildcard"):
                block_one = find2(ary[1], state)
                block_two = find2(ary[2], state)
                dif = [abs(block_one.x - block_two.x), abs(block_one.y - block_two.y)]
                dif = max(dif)
                if(ary[3] != "on-top-of"): dif = dif - 1
                if(dic.get(block_one.block_id, 0) == 0 and dic.get(block_two.block_id, 0) == 0):
                    cost = cost + dif
                    dic[block_one.block_id] = 1 
                    dic[block_two.block_id] = 1 
                    if(block_one.grabbed != True):
                        a = copy.deepcopy(block_one)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_one)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_two)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_two)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                                
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] != "wildcard"):
                block_two = find2(ary[2], state)
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(wildcards.get(ary[1], 0) == 0):
                            wildcards[ary[1]] = [m]
                        else:
                            wildcards[ary[1]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[1]]:
                    if(w != block_two):
                        dif = [abs(w.x - block_two.x), abs(w.y - block_two.y)]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                if(ary[3] != "on-top-of"): dif = dif - 1
                if(dic.get(block_two.block_id, 0) == 0 and dic.get(min_card[0].block_id, 0) == 0):
                    cost = cost + dif
                    dic[min_card[0].block_id] = 1 
                    dic[block_two.block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_two)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_two)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
                            
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] == "wildcard"):
                block_one = find2(ary[1], state)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(wildcards.get(ary[2], 0) == 0):
                            wildcards[ary[2]] = [m]
                        else:
                            wildcards[ary[2]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[2]]:
                    if(w != block_one):
                        dif = [abs(w.x - block_one.x), abs(w.y - block_one.y)]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                if(ary[3] != "on-top-of"): dif = dif - 1
                if(dic.get(block_one.block_id, 0) == 0 and dic.get(min_card[0].block_id, 0) == 0):
                    cost = cost + dif
                    dic[min_card[0].block_id] = 1 
                    dic[block_one.block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_one)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_one)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] == "wildcard"):
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(wildcards.get(ary[2], 0) == 0):
                            wildcards[ary[2]] = [m]
                        else:
                            wildcards[ary[2]].append(m)
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(wildcards.get(ary[1], 0) == 0):
                            wildcards[ary[1]] = [m]
                        else:
                            wildcards[ary[1]].append(m)
                min_card = (None, None, 1000)
                for w in wildcards[ary[1]]:
                    for v in wildcards[ary[2]]:
                        if(v != w):
                            dif = [abs(w.x - v.x), abs(w.y - v.y)]
                            dif = max(dif)
                            if(min_card[2] > dif):
                                min_card = (w, v, dif)
                if(ary[3] != "on-top-of"): dif = dif - 1
                if(dic.get(min_card[0].block_id, 0) == 0 and dic.get(min_card[1].block_id, 0) == 0):
                    cost = cost + dif
                    dic[min_card[0].block_id] = 1 
                    dic[min_card[1].block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(min_card[1].grabbed != True):
                        a = copy.deepcopy(min_card[1])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[1])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
    return cost

In [21]:
def h_2(state, goal):
    cost = 0
    dic = {}
    wildcards = {}
    tops = {}
    bottoms = {}
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 4):
                    for z in startState:
                        if(z.color == ary[3]):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
                if(len(x.split()) == 6):
                    if(wildcards.get(ary[1], 0) == 0):
                        for z in startState:
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
            if(ary[0] == "has" and ary[1][:8] != "wildcard" and len(x.split()) == 6 and dic.get(ary[1], 0) == 0):
                temp = find2(ary[1], state)
                dif = [abs(temp.x - int(ary[3])), abs(temp.y - int(ary[4])), abs(temp.z - int(ary[5]))]
                cost = cost + max(dif)
                if(temp.z != int(ary[5]) and not grabbed(state)):
                    cost = cost + 1
                if(a.grabbed != True):
                    a = copy.deepcopy(temp)
                    while(a.top != None and a.top.grabbed == False):
                        a = a.top
                        dic[a.block_id] = 1 
                    a = copy.deepcopy(temp)
                    while(a.below != None and a.below.grabbed == False):
                        a = a.below
                        dic[a.block_id] = 1
            if(ary[0] == "has" and ary[1][:8] == "wildcard" and len(x.split()) == 6):
                if(wildcards.get(ary[1], 0) != 0):
                    min_card = (None, 1000)
                    for w in wildcards[ary[1]]:
                        dif = [abs(temp.x - int(ary[3])), abs(temp.y - int(ary[4])), abs(temp.z - int(ary[5]))]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                    if(dic.get(min_card[0].block_id, 0) == 0):
                        cost = cost + min_card[1]
                    if(min_card[0].z != int(ary[5]) and not grabbed(state)):
                        cost = cost + 1
                        if(min_card[0].grabbed != True):
                            a = copy.deepcopy(min_card[0])
                            while(a.top != None and a.top.grabbed == False):
                                a = a.top
                                dic[a.block_id] = 1 
                            a = copy.deepcopy(min_card[0])
                            while(a.below != None and a.below.grabbed == False):
                                a = a.below
                                dic[a.block_id] = 1

                                
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] != "wildcard"):
                block_one = find2(ary[1], state)
                block_two = find2(ary[2], state)
                if(ary[3] == "on-top-of"):
                    dif = [abs(block_one.x - block_two.x), abs(block_one.y - block_two.y), abs(block_one.z + 1 - block_two.z)]
                else:
                    dif = [abs(block_one.x - block_two.x) - 1, abs(block_one.y - block_two.y) - 1, abs(block_one.z - block_two.z)]
                dif = max(dif)
                if(dic.get(block_one.block_id, 0) == 0 and dic.get(block_two.block_id, 0) == 0):
                    cost = cost + dif
                    if(ary[3] != "on-top-of" and block_one.z != block_two.z and not grabbed(state)):
                        cost = cost + 1
                    if(ary[3] == "on-top-of" and block_one.below != block_two and not grabbed(state)):
                        cost = cost + 1
                    dic[block_one.block_id] = 1 
                    dic[block_two.block_id] = 1 
                    if(block_one.grabbed != True):
                        a = copy.deepcopy(block_one)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_one)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_two)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_two)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                                
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] != "wildcard"):
                block_two = find2(ary[2], state)
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(wildcards.get(ary[1], 0) == 0):
                            wildcards[ary[1]] = [m]
                        else:
                            wildcards[ary[1]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[1]]:
                    if(w != block_two):
                        if(ary[3] == "on-top-of"):
                            dif = [abs(w.x - block_two.x), abs(w.y - block_two.y), abs(w.z + 1 - block_two.z)]
                        else:
                            dif = [abs(w.x - block_two.x) - 1, abs(w.y - block_two.y) - 1, abs(w.z - block_two.z)]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                if(dic.get(block_two.block_id, 0) == 0 and dic.get(min_card[0].block_id, 0) == 0):
                    cost = cost + min_card[1]
                    if(ary[3] != "on-top-of" and min_card[0].z != block_two.z and not grabbed(state)):
                        cost = cost + 1
                    if(ary[3] == "on-top-of" and min_card[0].below != block_two and not grabbed(state)):
                        cost = cost + 1
                    dic[min_card[0].block_id] = 1 
                    dic[block_two.block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_two)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_two)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
                            
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] == "wildcard"):
                block_one = find2(ary[1], state)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(wildcards.get(ary[2], 0) == 0):
                            wildcards[ary[2]] = [m]
                        else:
                            wildcards[ary[2]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[2]]:
                    if(w != block_one):
                        if(ary[3] == "on-top-of"):
                            dif = [abs(w.x - block_one.x), abs(w.y - block_one.y), abs(w.z + 1 - block_one.z)]
                        else:
                            dif = [abs(w.x - block_one.x) - 1, abs(w.y - block_one.y) - 1, abs(w.z - block_one.z)]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                if(dic.get(block_one.block_id, 0) == 0 and dic.get(min_card[0].block_id, 0) == 0):
                    cost = cost + min_card[1]
                    if(ary[3] != "on-top-of" and min_card[0].z != block_one.z and not grabbed(state)):
                        cost = cost + 1
                    if(ary[3] == "on-top-of" and block_one.below != min_card[0] and not grabbed(state)):
                        cost = cost + 1
                    dic[min_card[0].block_id] = 1 
                    dic[block_one.block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_one)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_one)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] == "wildcard"):
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [m]
                            else:
                                wildcards[ary[1]].append(m)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[2], 0) == 0):
                                wildcards[ary[2]] = [m]
                            else:
                                wildcards[ary[2]].append(m)
                min_card = (None, None, 1000)
                for w in wildcards[ary[1]]:
                    for v in wildcards[ary[2]]:
                        if(v != w):
                            if(ary[3] == "on-top-of"):
                                dif = [abs(w.x - v.x), abs(w.y - v.y), abs(w.z + 1 - v.z)]
                            else:
                                dif = [abs(w.x - v.x) - 1, abs(w.y - v.y) - 1, abs(w.z - v.z)]
                            dif = max(dif)
                            if(min_card[2] > dif):
                                min_card = (w, v, dif)              
                tops[min_card[0].block_id] = 1
                tops[min_card[1].block_id] = 1
                if(dic.get(min_card[0].block_id, 0) == 0 or dic.get(min_card[1].block_id, 0) == 0):
                    cost = cost + min_card[2]
                    g = get_grabbed(state)
                    if(ary[3] != "on-top-of" and min_card[0].z != min_card[1].z and (g[1] != min_card[0] and g[1] != min_card[1])):
                        cost = cost + 1
                    elif(ary[3] == "on-top-of" and min_card[0].below != min_card[1] and (g[1] != min_card[0] and g[1] != min_card[1])):
                        cost = cost + 1
                    dic[min_card[0].block_id] = 1 
                    dic[min_card[1].block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(min_card[1].grabbed != True):
                        a = copy.deepcopy(min_card[1])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[1])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
    return cost

In [22]:
class Node:
    def __init__(self, state, action, h=0, g=0 ,f=0):
        self.state = state
        self.action = action
        self.h = h
        self.g = g
        self.f = f
    def __repr__(self):
        return "Node(" + repr(self.state) + ", action=" + repr(self.action) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"
    def __str__(self):
        return "Node(" + repr(self.state) + ", action=" + repr(self.action) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

In [58]:
def h_3(state, goal):
    cost = 0
    dic = {}
    wildcards = {}
    tops = {}
    bottoms = {}
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 4):
                    for z in startState:
                        if(z.color == ary[3]):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
                if(len(x.split()) == 6):
                    if(wildcards.get(ary[1], 0) == 0):
                        for z in startState:
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
            if(ary[0] == "has" and ary[1][:8] != "wildcard" and len(x.split()) == 6 and dic.get(ary[1], 0) == 0):
                temp = find2(ary[1], state)
                dif = [abs(temp.x - int(ary[3])), abs(temp.y - int(ary[4])), abs(temp.z - int(ary[5]))]
                cost = cost + max(dif)
                if(temp.z != int(ary[5]) and not grabbed(state)):
                    cost = cost + 1
                if(a.grabbed != True):
                    a = copy.deepcopy(temp)
                    while(a.top != None and a.top.grabbed == False):
                        a = a.top
                        dic[a.block_id] = 1 
                    a = copy.deepcopy(temp)
                    while(a.below != None and a.below.grabbed == False):
                        a = a.below
                        dic[a.block_id] = 1
            if(ary[0] == "has" and ary[1][:8] == "wildcard" and len(x.split()) == 6):
                if(wildcards.get(ary[1], 0) != 0):
                    min_card = (None, 1000)
                    for w in wildcards[ary[1]]:
                        dif = [abs(temp.x - int(ary[3])), abs(temp.y - int(ary[4])), abs(temp.z - int(ary[5]))]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                    if(dic.get(min_card[0].block_id, 0) == 0):
                        cost = cost + min_card[1]
                    if(min_card[0].z != int(ary[5]) and not grabbed(state)):
                        cost = cost + 1
                        if(min_card[0].grabbed != True):
                            a = copy.deepcopy(min_card[0])
                            while(a.top != None and a.top.grabbed == False):
                                a = a.top
                                dic[a.block_id] = 1 
                            a = copy.deepcopy(min_card[0])
                            while(a.below != None and a.below.grabbed == False):
                                a = a.below
                                dic[a.block_id] = 1

                                
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] != "wildcard"):
                block_one = find2(ary[1], state)
                block_two = find2(ary[2], state)
                if(ary[3] == "on-top-of"):
                    dif = [abs(block_one.x - block_two.x), abs(block_one.y - block_two.y), abs(block_one.z + 1 - block_two.z)]
                else:
                    dif = [abs(block_one.x - block_two.x) - 1, abs(block_one.y - block_two.y) - 1, abs(block_one.z - block_two.z)]
                dif = max(dif)
                if(dic.get(block_one.block_id, 0) == 0 and dic.get(block_two.block_id, 0) == 0):
                    cost = cost + dif
                    if(ary[3] != "on-top-of" and block_one.z != block_two.z and not grabbed(state)):
                        cost = cost + 1
                    if(ary[3] == "on-top-of" and block_one.below != block_two and not grabbed(state)):
                        cost = cost + 1
                    dic[block_one.block_id] = 1 
                    dic[block_two.block_id] = 1 
                    if(block_one.grabbed != True):
                        a = copy.deepcopy(block_one)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_one)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_two)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_two)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                                
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] != "wildcard"):
                block_two = find2(ary[2], state)
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [m]
                            else:
                                wildcards[ary[1]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[1]]:
                    if(w != block_two):
                        if(ary[3] == "on-top-of"):
                            dif = [abs(w.x - block_two.x), abs(w.y - block_two.y), abs(w.z + 1 - block_two.z)]
                        else:
                            dif = [abs(w.x - block_two.x) - 1, abs(w.y - block_two.y) - 1, abs(w.z - block_two.z)]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                tops[min_card[1].block_id] = 1
                tops[block_two.block_id] = 1
                if(dic.get(block_two.block_id, 0) == 0 and dic.get(min_card[0].block_id, 0) == 0):
                    cost = cost + min_card[1]
                    dic[min_card[0].block_id] = 1 
                    dic[block_two.block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_two)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_two)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
                            
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] == "wildcard"):
                block_one = find2(ary[1], state)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[2], 0) == 0):
                                wildcards[ary[2]] = [m]
                            else:
                                wildcards[ary[2]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[2]]:
                    if(w != block_one):
                        if(ary[3] == "on-top-of"):
                            dif = [abs(w.x - block_one.x), abs(w.y - block_one.y), abs(w.z + 1 - block_one.z)]
                        else:
                            dif = [abs(w.x - block_one.x) - 1, abs(w.y - block_one.y) - 1, abs(w.z - block_one.z)]
                        dif = max(dif)
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                tops[min_card[1].block_id] = 1
                tops[block_one.block_id] = 1
                if(dic.get(block_one.block_id, 0) == 0 and dic.get(min_card[0].block_id, 0) == 0):
                    cost = cost + min_card[1]
                    dic[min_card[0].block_id] = 1 
                    dic[block_one.block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(block_two.grabbed != True):
                        a = copy.deepcopy(block_one)
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(block_one)
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] == "wildcard"):
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [m]
                            else:
                                wildcards[ary[1]].append(m)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[2], 0) == 0):
                                wildcards[ary[2]] = [m]
                            else:
                                wildcards[ary[2]].append(m)
                min_card = (None, None, 1000)
                dif = None
                for w in wildcards[ary[1]]:
                    for v in wildcards[ary[2]]:
                        if(v != w):
                            if(ary[3] == "on-top-of"):
                                dif = max([abs(w.x - v.x), abs(w.y - v.y), abs(w.z - 1 - v.z)])
                            else:
                                dif1 = max([abs(w.x + 1 - v.x), abs(w.y - v.y), abs(w.z - v.z)])
                                dif2 = max([abs(w.x - 1 - v.x), abs(w.y - v.y), abs(w.z - v.z)])
                                dif3 = max([abs(w.x - v.x), abs(w.y + 1 - v.y), abs(w.z - v.z)])
                                dif4 = max([abs(w.x - v.x), abs(w.y - 1 - v.y), abs(w.z - v.z)])
                                dif = min([dif1, dif2, dif3, dif4])

                            if(min_card[2] > dif):
                                
                                min_card = (w, v, dif)
  
                tops[min_card[0].block_id] = 1
                tops[min_card[1].block_id] = 1
                
                if(dic.get(min_card[0].block_id, 0) == 0 or dic.get(min_card[1].block_id, 0) == 0 and min_card[2] > 0):
                    cost = cost + min_card[2]
                    dic[min_card[0].block_id] = 1 
                    dic[min_card[1].block_id] = 1 
                    if(min_card[0].grabbed != True):
                        a = copy.deepcopy(min_card[0])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[0])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                    if(min_card[1].grabbed != True):
                        a = copy.deepcopy(min_card[1])
                        while(a.top != None and a.top.grabbed == False):
                            a = a.top
                            dic[a.block_id] = 1 
                        a = copy.deepcopy(min_card[1])
                        while(a.below != None and a.below.grabbed == False):
                            a = a.below
                            dic[a.block_id] = 1
                            
    return cost

In [65]:
def h_4(state, goal):
    cost = 0
    dic = {}
    wildcards = {}
    tops = {}
    bottoms = {}
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 4):
                    for z in startState:
                        if(z.color == ary[3]):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
                if(len(x.split()) == 6):
                    if(wildcards.get(ary[1], 0) == 0):
                        for z in startState:
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
            if(ary[0] == "has" and ary[1][:8] != "wildcard" and len(x.split()) == 6 and dic.get(ary[1], 0) == 0):
                temp = find2(ary[1], state)
                dif = abs(temp.x - int(ary[3])) + abs(temp.y - int(ary[4])) + abs(temp.z - int(ary[5]))
                cost = cost + dif
            if(ary[0] == "has" and ary[1][:8] == "wildcard" and len(x.split()) == 6):
                if(wildcards.get(ary[1], 0) != 0):
                    min_card = (None, 1000)
                    for w in wildcards[ary[1]]:
                        dif = abs(w.x - int(ary[3])) + abs(w.y - int(ary[4])) + abs(w.z - int(ary[5]))
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                    cost = cost + min_card[1]
                                
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] != "wildcard"):
                block_one = find2(ary[1], state)
                block_two = find2(ary[2], state)
                if(ary[3] == "on-top-of"):
                    dif = abs(block_one.x - block_two.x) + abs(block_one.y - block_two.y) + abs(block_one.z + 1 - block_two.z)
                else:
                    dif1 = abs(w.x + 1 - v.x) + abs(w.y - v.y) + abs(w.z - v.z)
                    dif2 = abs(w.x - 1 - v.x) + abs(w.y - v.y) + abs(w.z - v.z)
                    dif3 = abs(w.x - v.x) + abs(w.y + 1 - v.y) + abs(w.z - v.z)
                    dif4 = abs(w.x - v.x) + abs(w.y - 1 - v.y) + abs(w.z - v.z)
                    dif = min([dif1, dif2, dif3, dif4])
                cost = cost + dif
                                
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] != "wildcard"):
                block_two = find2(ary[2], state)
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [m]
                            else:
                                wildcards[ary[1]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[1]]:
                    if(w != block_two):
                        if(ary[3] == "on-top-of"):
                            dif = abs(w.x - block_two.x) + abs(w.y - block_two.y) + abs(w.z + 1 - block_two.z)
                        else:
                            dif1 = abs(w.x + 1 - block_two.x) + abs(w.y - block_two.y) + abs(w.z - block_two.z)
                            dif2 = abs(w.x - 1 - block_two.x) + abs(w.y - block_two.y) + abs(w.z - block_two.z)
                            dif3 = abs(w.x - block_two.x) + abs(w.y + 1 - block_two.y) + abs(w.z - block_two.z)
                            dif4 = abs(w.x - block_two.x) + abs(w.y - 1 - block_two.y) + abs(w.z - block_two.z)
                            dif = min([dif1, dif2, dif3, dif4])
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                cost = cost + min_card[1]                          
                            
            if(ary[0] == "is" and ary[1][:8] != "wildcard" and ary[2][:8] == "wildcard"):
                block_one = find2(ary[1], state)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[2], 0) == 0):
                                wildcards[ary[2]] = [m]
                            else:
                                wildcards[ary[2]].append(m)
                min_card = (None, 1000)
                for w in wildcards[ary[2]]:
                    if(w != block_one):
                        if(ary[3] == "on-top-of"):
                            dif = abs(block_one.x - w.x) + abs(block_one.y - w.y) + abs(block_one.z + 1 - w.z)
                        else:
                            dif1 = abs(block_one.x + 1 - w.x) + abs(block_one.y - w.y) + abs(block_one.z - w.z)
                            dif2 = abs(block_one.x - 1 - w.x) + abs(block_one.y - w.y) + abs(block_one.z - w.z)
                            dif3 = abs(block_one.x - w.x) + abs(block_one.y + 1 - w.y) + abs(block_one.z - w.z)
                            dif4 = abs(block_one.x - w.x) + abs(block_one.y - 1 - w.y) + abs(block_one.z - w.z)
                            dif = min([dif1, dif2, dif3, dif4])
                        if(min_card[1] > dif):
                            min_card = (w, dif)
                cost = cost + min_card[1]
                            
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] == "wildcard"):
                if(wildcards.get(ary[1], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [m]
                            else:
                                wildcards[ary[1]].append(m)
                if(wildcards.get(ary[2], 0) == 0):
                    for m in state:
                        if(tops.get(m.block_id, 0) == 0):
                            if(wildcards.get(ary[2], 0) == 0):
                                wildcards[ary[2]] = [m]
                            else:
                                wildcards[ary[2]].append(m)
                min_card = (None, None, 1000)
                dif = None
                for w in wildcards[ary[1]]:
                    for v in wildcards[ary[2]]:
                        if(v != w):
                            if(ary[3] == "on-top-of"):
                                dif = abs(w.x - v.x) + abs(w.y - v.y) + abs(w.z - 1 - v.z)
                            else:
                                dif1 = abs(w.x + 1 - v.x) + abs(w.y - v.y) + abs(w.z - v.z)
                                dif2 = abs(w.x - 1 - v.x) + abs(w.y - v.y) + abs(w.z - v.z)
                                dif3 = abs(w.x - v.x) + abs(w.y + 1 - v.y) + abs(w.z - v.z)
                                dif4 = abs(w.x - v.x) + abs(w.y - 1 - v.y) + abs(w.z - v.z)
                                dif = min([dif1, dif2, dif3, dif4])
                            if(min_card[2] > dif):
                                min_card = (w, v, dif)
  
                cost = cost + min_card[2]
    return cost

In [116]:
def aStarLow(parentNode, goalState, actionsF, take_actionF, hF, fmax):
    if(check(parentNode.state, goalState)):
        return([parentNode.action], parentNode.g)
    actions = actionsF(parentNode.state)
    if not actions:
        return("no more moves", float("inf"))
    children = []
    for action in actions:
        childState = take_actionF(action, parentNode.state)
        h = hF(childState, goalState)
        g = parentNode.g + action[4]
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, action = action, h=h, g=g, f=f)
        children.append(childNode)
    while True:
        minChild = min(children, key = lambda x: x.f)
        print(minChild.f)
        print(minChild.state)
        print(hF(minChild.state, goalState))
        if minChild.f > fmax:
            return ("max steps exceded", minChild.f)
        alternativef = minChild.f if len(children) > 1 else float('inf')
        result, minChild.f = aStarLow(minChild, goalState, actionsF, take_actionF, hF, min(fmax,alternativef))
        if result is not "no more moves" and result is not "max steps exceded":         
            result.insert(0, parentNode.action) 
            return (result, minChild.f) 
        
def aStarSearchLow(startState, goalState, actionsF, take_actionF, hF, fmax):
    answer = aStarLow(Node(state=startState, action=None, f=0, g=0, h=0), goalState, actionsF, take_actionF, hF, fmax)
    if((answer[0] ==  "max steps exceded") or (answer[0] == "no more moves")):
        print(answer)
    else:
        print(answer[0][1::])
        for a in answer[0][1::]:
            if("-r" in a[0]):
                print("(release, " + str(a[2][1]) + ")")
            if("-g" in a[0]):
                print("(grab, " + str(a[1][1]) + ")")
            print("(" + str(a[0])[:5] + ", " + str(a[1][1]) + ", " + str(a[3]) + ")")
        print(answer[1])

In [117]:
def actionsH(state, goalState):
    wildcards = {}
    action = []
    actions = []
    cost = 1
    
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 4):
                    action.append(copy.deepcopy(x))
                    for z in startState:
                        if(z.color == ary[3]):
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
    for x in goalState:
        if(len(x.split()) == 4 or len(x.split()) == 6):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has" and ary[1][:8] == "wildcard"):
                if(len(x.split()) == 6):
                    if(wildcards.get(ary[1], 0) == 0):
                        for z in startState:
                            if(wildcards.get(ary[1], 0) == 0):
                                wildcards[ary[1]] = [z]
                            else:
                                wildcards[ary[1]].append(z)
                    for w in wildcards[ary[1]]:
                        manhattan = abs(w.x - int(ary[3])) + abs(w.y - int(ary[4])) + abs(w.z - int(ary[5]))
                        if(manhattan > 0):
                            actions.append(action + [x, manhattan])
                        
            if(ary[0] == "is" and ary[1][:8] == "wildcard" and ary[2][:8] == "wildcard"):
                if(wildcards.get(ary[1], 0) == 0):
                    for z in startState:
                        if(wildcards.get(ary[1], 0) == 0):
                            wildcards[ary[1]] = [z]
                        else:
                            wildcards[ary[1]].append(z)                
                if(wildcards.get(ary[2], 0) == 0):
                    for z in startState:
                        if(wildcards.get(ary[2], 0) == 0):
                            wildcards[ary[2]] = [z]
                        else:
                            wildcards[ary[2]].append(z) 
                for w in wildcards[ary[1]]:
                    for v in wildcards[ary[2]]:
                        if(w != v):
                            if(ary[3] == "on-top-of"):
                                manhattan = abs(w.x - v.x) + abs(w.y - v.y) + abs(w.z - (v.x + 1))
                                if(manhattan > 0):
                                    actions.append(action + [x, manhattan])
                            elif(ary[3] == "side-by-side"):
                                manhattan1 = (abs(w.x - (v.x + 1)) + abs(w.y - v.y) + abs(w.z - v.z), (v.x + 1, v.y, v.z))
                                manhattan2 = (abs(w.x - (v.x - 1)) + abs(w.y - v.y) + abs(w.z - v.z), (v.x - 1, v.y, v.z))
                                manhattan3 = (abs(w.x - v.x) + abs(w.y - (v.y + 1)) + abs(w.z - v.z), (v.x, v.y + 1, v.z))
                                manhattan4 = (abs(w.x - v.x) + abs(w.y - (v.y - 1)) + abs(w.z - v.z), (v.x + 1, v.y - 1, v.z))
                                manhattan = [manhattan1, manhattan2, manhattan3, manhattan4]
                                minManhattan = min(manhattan, key = lambda x: x[0])
                                if(minManhattan[0] > 0):
                                    actions.append(action + [x, minManhattan[0]])
#                                 manhattan1 = (abs((w.x + 1) - v.x) + abs(w.y - v.y) + abs(w.z - v.z), (w.x + 1, w.y, w.z))
#                                 manhattan2 = (abs((w.x - 1) - v.x) + abs(w.y - v.y) + abs(w.z - v.z), (w.x - 1, w.y, w.z))
#                                 manhattan3 = (abs(w.x - v.x) + abs((w.y + 1) - v.y) + abs(w.z - v.z), (w.x, w.y + 1, w.z))
#                                 manhattan4 = (abs(w.x - v.x) + abs((w.y - 1) - v.y) + abs(w.z - v.z), (w.x, w.y - 1, w.z))
#                                 manhattan = [manhattan1, manhattan2, manhattan3, manhattan4]
#                                 minManhattan = min(manhattan, key = lambda x: x[0])
#                                 if(minManhattan[0] > 0):
#                                     actions.append(("side", v.block_id, (minManhattan[1][0], minManhattan[1][1], minManhattan[1][2]), minManhattan[0]))
                                
    
    return actions

In [118]:
def take_actionH(action, state):
    newState = copy.deepcopy(state)
    if(action[0] == "locate"):
        find(action[2])
        slide(newState[action[1][0]], action[3][0], action[3][1], newState)
    elif(action[0] == "slide-r"):
        release(newState[action[2][0]], newState)
        slide(newState[action[1][0]], action[3][0], action[3][1], newState)
    elif(action[0] == "carry"):
        carry(newState[action[1][0]], action[3][0], action[3][1], action[3][2], newState)
    elif(action[0] == "carry-g"):
        grab(newState[action[1][0]], newState)
        carry(newState[action[1][0]], action[3][0], action[3][1], action[3][2], newState)
    elif(action[0] == "carry-rg"):
        release(newState[action[2][0]], newState)
        grab(newState[action[1][0]], newState)
        carry(newState[action[1][0]], action[3][0], action[3][1], action[3][2], newState)
 
    return newState

In [120]:
startState = read_startState("input3.txt")
goalState = read_goalState("output3.txt") #Output File
# print(goalState)
actionsH(startState, goalState)
# actions(startState)
for x in actionsH(startState, goalState):
#     print(x[:-1])
#     print(h_4(startState, x[:-1]))
#     slide(startState[0], 1, 1, startState)
#     print(h_4(startState, x[:-1]))
#     slide(startState[0], 0, 1, startState)
#     print(h_4(startState, x[:-1]))
#     slide(startState[0], 0, 1, startState)
#     print(h_4(startState, x[:-1]))
#     slide(startState[0], 0, 1, startState)
#     print(h_4(startState, x[:-1]))
#     print(startState)

    aStarSearchLow(startState, x[:-1], actions, take_action, h_4, 1000)

1
[{block1, (0, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
1
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
1
[{

2
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (1, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{

3
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (1, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (2, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{

3
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (6, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (4, 6, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (0, 3, 0

3
[{block1, (1, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (6, 6, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
2
[{block1, (2, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (6, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (1, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (6, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
2
[{block1, (2, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (1, 3, 0

...

[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (4, 4, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
4
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (4, 4, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (3, 4, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
4
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (3, 4, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 6, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side:

3
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
4
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (1, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
4
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (5, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (2, 3, 0), (color: red), (on-top-of: None), (side-by-sid

4
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (6, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (4, 6, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (6, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
4
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (4, 6, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (6, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
3
[{block1, (1, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 6, 0), (color: green), (on-top-of: None), (side-by-side: [])}
, {block3, (6, 7, 0), (color: blue), (on-top-of: None), (side-by-side: [])}
]
5
4
[{block1, (0, 3, 0), (color: red), (on-top-of: None), (side-by-sid

4
[{block1, (0, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (2, 3, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
4
[{block1, (1, 4, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
3
[{block1, (2, 2, 0), (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (5, 5, 0), (color: green), (on-top-of: None), (side-by-side: ['block3'])}
, {block3, (4, 5, 0), (color: blue), (on-top-of: None), (side-by-side: ['block2'])}
]
5
4
[{

KeyboardInterrupt: 

In [101]:
-

In [102]:
startState = read_startState("input3.txt")
goalState = read_goalState("output3.txt") #Output File
aStarSearchLow(startState, goalState, actions, take_action, h_4, 1000)


1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2


KeyboardInterrupt: 

In [ ]:
startState = read_startState("input2.txt")
goalState = read_goalState("output2.txt") #Output File

print(startState)
print(goalState)
print(h_2(startState, goalState))

grab(startState[0], startState)
print(h_2(startState, goalState))
carry(startState[0], 1, 1, 1, startState)
print(h_2(startState, goalState))
carry(startState[0], 1, 1, 0, startState)
print(h_2(startState, goalState))
print(startState)
release(startState[0],startState)
print(h_2(startState, goalState))
grab(startState[2], startState)
print(h_2(startState, goalState))
# carry(startState[2], -1, -1, 1, startState)
# print(h_2(startState, goalState))
# carry(startState[2], -1, -1, 1, startState)
# print(h_2(startState, goalState))
# print(startState)
aStarSearch(startState, goalState, actions, take_action, h_2, 1000)
# FIGURE OUT TOPS AND BOTTOMS